# Record Linkage

In [7]:
import recordlinkage
import pandas as pd
import collections
import time
import os

In [8]:
MEDIATED_DATASETS_PATH = "./Mediated Datasets/"

In [9]:
def get_all_files(path_dir):
    res = []
    for path in os.listdir(path_dir):
        if os.path.isfile(os.path.join(path_dir, path)) and path != '.DS_Store':
            res.append(path)
    return res


def get_features(df_l, df_r, name_threshold, is_name):
    feature_value = 0
    columns_l = df_l.columns.values.tolist()
    columns_r = df_r.columns.values.tolist()
    # set_index
    indexer = recordlinkage.Index()
    indexer.full()
    candidate_links = indexer.index(df_l, df_r)

    # Comparison step
    compare_cl = recordlinkage.Compare()
    compare_cl.string("name", "name", method="levenshtein", threshold=name_threshold, label="name")

    if not is_name:
        if 'ceo' in columns_l and 'ceo' in columns_r:
            compare_cl.string("ceo", "ceo", method="levenshtein", threshold=0.85, label="ceo")
            feature_value = 1

    return compare_cl.compute(candidate_links, df_l, df_r), feature_value


def get_pairs(features, feature_value):
    # Classification step
    pairs = features[features.sum(axis=1) > feature_value]
    return pairs['name'].keys().to_list()


def rename_columns(columns_l, columns_r):
    columns_l = [c + '_l' for c in columns_l]
    columns_r = [c + '_r' for c in columns_r]
    return columns_l, columns_r

def remove_suffix(columns):
    return [c[:-2] for c in columns]


def find_duplicated_columns(columns_l, columns_r):
    attr_cleaned = columns_l + columns_r
    duplicates = [item for item, count in collections.Counter(attr_cleaned).items() if count > 1]
    return duplicates


def join(matching_dataset_left, matching_dataset_right):
    duplicates = find_duplicated_columns(matching_dataset_left.columns.values.tolist(), matching_dataset_right.columns.values.tolist())

    column_left, column_right = rename_columns(matching_dataset_left.columns.values.tolist(),
                                           matching_dataset_right.columns.values.tolist())
    matching_dataset_left.columns = column_left
    matching_dataset_right.columns = column_right
    joined_df = pd.concat([matching_dataset_left, matching_dataset_right], axis=1)
    candidate_joined_df = joined_df.copy()

    for col in duplicates:
        candidates = joined_df[[col + '_l', col + '_r']]
        candidate_labels = candidates.columns.values.tolist()
        #display(candidates.sample(10))
        # time.sleep(1)
        #idx_drop = int(input("Inserisci l'indice (0 o 1) della colonna che vuoi scartare nel dataset finale: "))
        joined_df.drop(candidate_labels[0], axis=1, inplace=True)

    joined_df.columns = [col[:-2] for col in joined_df.columns.values.tolist()]
    return joined_df, candidate_joined_df

In [10]:

def prova(df_l, df_r):
    features, feature_value = get_features(df_l, df_r, 0.95, False)
    pairs = get_pairs(features, feature_value)
    features.sum(axis=1).value_counts().sort_index(ascending=False)

    left = []
    right = []

    for elem in pairs:
        left.append(df_l.loc[elem[0]])
        right.append(df_r.loc[elem[1]])

    matching_dataset_left = pd.DataFrame(left)
    matching_dataset_right = pd.DataFrame(right)
    matching_dataset_left = matching_dataset_left.reset_index(drop=True)
    matching_dataset_right = matching_dataset_right.reset_index(drop=True)

    difference_l = pd.concat([df_l, matching_dataset_left]).drop_duplicates(keep=False)
    difference_r = pd.concat([df_r, matching_dataset_right]).drop_duplicates(keep=False)

    match_join_df, candidate_join_df = join(matching_dataset_left, matching_dataset_right)
    display(match_join_df)


    features_difference, feature_value_difference = get_features(difference_l, difference_r, 0.95, True)
    pairs_difference = get_pairs(features_difference, feature_value_difference)
    features_difference.sum(axis=1).value_counts().sort_index(ascending=False)

    left = []
    right = []

    for elem in pairs_difference:
        left.append(difference_l.loc[elem[0]])
        right.append(difference_r.loc[elem[1]])

    matching_difference_dataset_left = pd.DataFrame(left)
    matching_difference_dataset_right = pd.DataFrame(right)
    matching_difference_dataset_left = matching_difference_dataset_left.reset_index(drop=True)
    matching_difference_dataset_right = matching_difference_dataset_right.reset_index(drop=True)

    difference_join_df, difference_candidate_join_df = join(matching_difference_dataset_left, matching_difference_dataset_right)
    display(difference_join_df)

    result= pd.concat([match_join_df, difference_join_df])
    display(result)

    matching_difference_dataset_left.columns = remove_suffix(matching_difference_dataset_left.columns.values.tolist())
    matching_difference_dataset_right.columns = remove_suffix(matching_difference_dataset_right.columns.values.tolist())

    unique_left_df =pd.concat([difference_l, matching_difference_dataset_left]).drop_duplicates(keep=False)
    result = pd.concat([result, unique_left_df])

    unique_right_df = pd.concat([difference_r, matching_difference_dataset_right]).drop_duplicates(keep=False)
    result = pd.concat([result, unique_right_df])

    result = result.reset_index(drop=True)

    return result


files = get_all_files(MEDIATED_DATASETS_PATH)
df_l = pd.read_json(MEDIATED_DATASETS_PATH + files[0], encoding='utf-8', lines=True, dtype=object)
files.pop(0)
for file in files:
        df_r = pd.read_json(MEDIATED_DATASETS_PATH + file, encoding='utf-8', lines=True, dtype=object)
        df_l = prova(df_l, df_r)
df_l

,name,headquarters,country,continent,region,sub_region,founded,employees,ceo,market_cap,...,industry,revenue,link,share_price,results_for_year,total_equity,founders,market_value,address,type
0,johnson & johnson,"newbrunswick,nj,unitedstates",united states,,,,january 1886,"141,700",joaquin duato,475930000000,...,drug manufacturersgeneral,,,,,,,,,
1,morgan stanley,"newyork,ny,unitedstates",united states,,,,1935,"75,000",james patrick gorman,164050000000,...,capital markets,,,,,,,,,
2,sanofi,"paris,france",france,,,,1973,"95,442",paul hudson,132289999999,...,drug manufacturersgeneral,,,,,,,,,
3,conocophillips,"houston,tx,unitedstates",united states,,,,30 august 2002,"9,900",ryan m lance,136320000000,...,oil & gas e&p,,,,,,,,,
4,petroleo brasileiro sa - petrobras,"riodejaneiro,rj,brazil",brazil,,,,3 october 1953,"46,416",joaquim silva e luna,98370000000,...,oil & gas integrated,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,capitec bank holdings ltd,"stellenbosch,southafrica",south africa,,,,,"14,789",gerhardus metselaar fourie bcom mba bcomm hons...,18210000000,...,banks,,,,,,,,,
681,asahi group holdings ltd,"tokyo,japan",japan,,,,,"30,020",atsushi katsuki,18210000000,...,beveragesbrewers,,,,,,,,,
682,catalent inc,"somerset,nj,unitedstates",united states,,,,2007,"17,300",john r chiminski,18240000000,...,drug manufacturersspecialty & generic,,,,,,,,,
683,quanta services inc,"houston,tx,unitedstates",united states,,,,1997,"43,700",earl c austin jr,18250000000,...,engineering & construction,,,,,,,,,


,name,headquarters,country,continent,region,sub_region,founded,employees,ceo,market_cap,...,industry,revenue,link,share_price,results_for_year,total_equity,founders,market_value,address,type
0,china life insurance company limited,"beijing,china",china,,,,1949,"104,160",,97310000000,...,insurancelife,,,,,,,,,
1,nippon telegraph & telephone corp,"tokyo,japan",japan,,,,,"324,667",jun sawada,103750000000,...,telecom services,,,,,,,,,
2,al rajhi banking and investment corp,"riyadh,saudiarabia",saudi arabia,,,,,"15,078",,106370000000,...,banks,,,,,,,,,
3,vale sa,"riodejaneiro,rj,brazil",brazil,,,,,,,107990000000,...,other industrial metals & mining,,,,,,,,,
4,anheuser-busch inbev sa/nv,"leuven,belgium",belgium,,,,,"169,339",,123590000000,...,beveragesbrewers,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,astra international tbk pt,"jakarta,indonesia",indonesia,,,,,"123,894",,18900000000,...,auto parts,,,,,,,,,
184,nice ltd,"raaanana,israel",israel,,,,1986,"6,800",,19120000000,...,softwareapplication,,,,,,,,,
185,zhejiang huayou cobalt co ltd,"tongxiang,china",china,,,,,"8,079",,19200000000,...,other industrial metals & mining,,,,,,,,,
186,restaurant brands international inc,"toronto,on,canada",canada,,,,"december 15, 2014","5,700",,18190000000,...,restaurants,,,,,,,,,


,name,headquarters,country,continent,region,sub_region,founded,employees,ceo,market_cap,...,industry,revenue,link,share_price,results_for_year,total_equity,founders,market_value,address,type
0,johnson & johnson,"newbrunswick,nj,unitedstates",united states,,,,january 1886,"141,700",joaquin duato,475930000000,...,drug manufacturersgeneral,,,,,,,,,
1,morgan stanley,"newyork,ny,unitedstates",united states,,,,1935,"75,000",james patrick gorman,164050000000,...,capital markets,,,,,,,,,
2,sanofi,"paris,france",france,,,,1973,"95,442",paul hudson,132289999999,...,drug manufacturersgeneral,,,,,,,,,
3,conocophillips,"houston,tx,unitedstates",united states,,,,30 august 2002,"9,900",ryan m lance,136320000000,...,oil & gas e&p,,,,,,,,,
4,petroleo brasileiro sa - petrobras,"riodejaneiro,rj,brazil",brazil,,,,3 october 1953,"46,416",joaquim silva e luna,98370000000,...,oil & gas integrated,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,astra international tbk pt,"jakarta,indonesia",indonesia,,,,,"123,894",,18900000000,...,auto parts,,,,,,,,,
184,nice ltd,"raaanana,israel",israel,,,,1986,"6,800",,19120000000,...,softwareapplication,,,,,,,,,
185,zhejiang huayou cobalt co ltd,"tongxiang,china",china,,,,,"8,079",,19200000000,...,other industrial metals & mining,,,,,,,,,
186,restaurant brands international inc,"toronto,on,canada",canada,,,,"december 15, 2014","5,700",,18190000000,...,restaurants,,,,,,,,,


,name,headquarters,country,continent,region,sub_region,founded,employees,ceo,market_cap,...,industry,revenue,link,share_price,results_for_year,total_equity,founders,market_value,address,type
0,johnson & johnson,"newbrunswick,nj,unitedstates",united states,,,,january 1886,"141,700",joaquin duato,475930000000,...,drug manufacturersgeneral,,,,,,,,,
1,morgan stanley,"newyork,ny,unitedstates",united states,,,,1935,"75,000",james patrick gorman,164050000000,...,capital markets,,,,,,,,,
2,sanofi,"paris,france",france,,,,1973,"95,442",paul hudson,132289999999,...,drug manufacturersgeneral,,,,,,,,,
3,conocophillips,"houston,tx,unitedstates",united states,,,,30 august 2002,"9,900",ryan m lance,136320000000,...,oil & gas e&p,,,,,,,,,
4,petroleo brasileiro sa - petrobras,"riodejaneiro,rj,brazil",brazil,,,,3 october 1953,"46,416",joaquim silva e luna,98370000000,...,oil & gas integrated,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,hdfc bank limited,"mumbai,india",india,,,,,"134,412",sashidhar jagdishan,115680000000,...,banks,,,,,,,,,
1123,blackrock inc,"newyork,ny,unitedstates",united states,,,,1988,"18,400",laurence douglas fink,115980000000,...,asset management,,,,,,,,,
1124,unilever plc,"london,unitedkingdom",united kingdom,,,,2 september 1929,"148,012",alan w jope,117840000000,...,household & personal products,,,,,,,,,
1125,applied materials inc,"santaclara,ca,unitedstates",united states,,,,"november 10, 1967","28,500",gary e dickerson,121120000000,...,semiconductor equipment & materials,,,,,,,,,


In [5]:
files = get_all_files(MEDIATED_DATASETS_PATH)
df_l = pd.read_json(MEDIATED_DATASETS_PATH + files[0], encoding='utf-8', lines=True, dtype=object)
files.pop(0)
for file in files:
        df_r = pd.read_json(MEDIATED_DATASETS_PATH + file, encoding='utf-8', lines=True, dtype=object)
        df_l = prova(df_l, df_r)
df_l

KeyboardInterrupt: 